In [ ]:
# The purpose of this code is to develop a GIF with a map of Minnesota (MN) successively adding MN state highways-just for fun

In [1]:
import pandas as pd
import geopandas as gpd
import descartes
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry
import imageio # to make the GIF file
import os
from progressbar import ProgressBar
import warnings

In [2]:
roads = 'STREETS_LOAD.shp'

In [3]:
states = '/Users/matthewwilliamhaas/Documents/shapefile_data_sources/cb_2018_us_state_20m/cb_2018_us_state_20m.shp'
map_states = gpd.read_file(states)

In [4]:
map_states[map_states.STATEFP == '27']

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
28,27,00662849,0400000US27,27,MN,Minnesota,00,206228939448,18945217189,"POLYGON ((-97.22904 49.00069, -96.93096 48.999..."


In [5]:
MN = map_states[(map_states.STATEFP == '27')]
MN = MN.to_crs({'init' : 'epsg:26915'})

In [6]:
# read in data
road_data = gpd.read_file(roads)

In [69]:
# used this code for the map of MN and WI--not sure if it will be useful here or not
#TH61 = TH61.to_crs({"init" : "epsg:26915"})

In [7]:
# Convert code for making one-off highway maps to a function to do in a high-throughput manner.
# Needed arguments: 1) path to image (highway sign), 2) indicator of current highway to plot geodata, and 3) output filename

# Initiate empty list to hold highways that have already been plotted
#alreadyPlotted = [] # Keep outside function definitation so it doesn't get cleared every time the function is called
#geoDataAll = gpd.GeoSeries(crs='epsg:26915') # make empty geopandas geoseries dataframe

def MakeHwyMap (imagefile, currentHwy, outfile):
    global geoDataAll
    data = road_data.STREET_NAM == currentHwy
    geoData = road_data[(data)].geometry
    im = plt.imread(imagefile)
    fig, ax = plt.subplots(1, figsize = (20,15))
    MN.plot(color = 'white', linewidth = 1.0, ax = ax, edgecolor = 'black')
    plt.title('Minnesota state ("trunk") highway system', fontsize = 60)
    #if currentHwy in alreadyPlotted:
    geoDataAll.plot(color ='grey', linewidth = 3.0, ax = ax, edgecolor = 'black')
    #else:
    geoData.plot(color ='red', linewidth = 3.0, ax = ax, edgecolor = 'black')
    #alreadyPlotted.append(currentHwy)
    geoDataAll = geoDataAll.append(geoData)
    newax = fig.add_axes([0.8, 0.5, 0.2, 0.3], anchor = 'NE', zorder = 1)
    newax.imshow(im)
    ax.axis('off') # turn off the axis
    newax.axis('off')
    plt.savefig(outfile)
    plt.close('all')

In [11]:
infile = open('minnesota_highway_list.csv', 'r', encoding='utf-8-sig') # open file
highways = infile.readlines()  # begin reading data

out_PNG_list = [] # make empty list to hold names of output files to make GIF at end
#alreadyPlotted = []
geoDataAll = gpd.GeoSeries(crs='epsg:26915') # make empty geopandas geoseries dataframe

pbar = ProgressBar()

# Run the MakeHwyMap function for each line in the file (each line represents a highway)
for line in pbar(highways):
    line = line.rstrip()
    line = line.split(',')
    warnings.filterwarnings('ignore') # silence warnings
    MakeHwyMap(imagefile = line[0], currentHwy = line[1], outfile = line[2])
    out_PNG_list.append(line[2])

100% |########################################################################|


In [12]:
# make the GIF file
with imageio.get_writer('movie.gif', mode = 'I', duration = 1) as writer:
    for filename in out_PNG_list:
        image = imageio.imread(filename)
        writer.append_data(image)

In [344]:
road_data[(road_data.STREET_NAM == 'Division Ave')] # MN TH 120 also goes by Century Ave, Geneva Ave, Division, and more

,STREET_NAM,STREET_PRE,BASE_NAME,STREET_TYP,STREET_SUF,E_911,ROUTE_SYST,ROUTE_NUMB,TIS_CODE,DIVIDED_ON,TRAFFIC_DI,SHAPE_Leng,geometry
14646,Division Ave,None,Division,Ave,None,U,10,28,1036300028,T,B,55.021205,"LINESTRING (549431.000 4837178.500, 549432.563..."
23747,Division Ave,None,Division,Ave,None,U,07,151,0762000151,T,B,299.758187,"LINESTRING (499221.907 4993821.500, 499227.000..."
62975,Division Ave,None,Division,Ave,None,U,08,2009,0831002009,T,B,71.672287,"LINESTRING (450339.428 5287088.761, 450338.048..."
79974,Division Ave,None,Division,Ave,None,U,10,26,1003300026,T,B,403.282540,"LINESTRING (449943.054 5287574.953, 450346.318..."
102822,Division Ave,None,Division,Ave,None,U,07,236,0731000236,T,B,785.642727,"LINESTRING (450373.719 5288373.501, 450375.499..."
110677,Division Ave,None,Division,Ave,None,U,05,125,0540900125,T,B,122.499294,"LINESTRING (499190.407 4993247.001, 499189.844..."
158291,Division Ave,None,Division,Ave,None,U,05,125,0540900125,T,B,105.508973,"LINESTRING (499189.405 4992556.500, 499190.781..."
213530,Division Ave,None,Division,Ave,None,U,10,12,1003250012,T,B,114.515863,"LINESTRING (440263.625 5337956.000, 440265.531..."
236799,Division Ave,None,Division,Ave,None,U,05,125,0540900125,T,B,111.018579,"LINESTRING (499190.562 4992771.000, 499188.531..."
239975,Division Ave,None,Division,Ave,None,U,10,28,1036300028,T,B,101.700839,"LINESTRING (549427.875 4836980.000, 549428.288..."


In [360]:
pip install progressbar

  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12074 sha256=42624e65a4c3390159a874f8d2247b7ac6247c4dc43fe5afd663ac270e5ff515
  Stored in directory: /Users/matthewwilliamhaas/Library/Caches/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar
Note: you may need to restart the kernel to use updated packages.
